# 行情数据爬虫

1. tushare对爬虫进行了封装，目前使用需要账号积分，越来越不便利；
2. 使用爬虫爬取雪球网站的股票日行情信息；
3. 爬虫原理：简单来说分为两步
    1. 发出请求获取服务器返回信息；
    2. 对返回信息解析整理；

# 获取请求URL

https://stock.xueqiu.com/v5/stock/chart/kline.json?symbol=SZ000001&begin=1572879616012&period=day&type=before&count=-160&indicator=kline,pe,pb,ps,pcf,market_capital,agt,ggt,balance

# 环境配置

In [1]:
import requests
import json
import pandas as pd
import datetime
import numpy  as np

# pyecharts
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Kline, Line, Bar, Grid,Scatter

# 请求数据

In [2]:
res=requests.get(
    url='https://stock.xueqiu.com/v5/stock/chart/kline.json?symbol=SZ000001&begin=1572879616012&period=day&type=before&count=-160&indicator=kline,pe,pb,ps,pcf,market_capital,agt,ggt,balance',
    headers={
        'User-Agent':'pyqi',
        'Cookie':'xq_a_token=1d1413ef0b90ad65cb650d1d8651ff2941335e2e',
    },
)

In [3]:
res

<Response [200]>

In [4]:
data=json.loads(res.text)

In [5]:
data.keys()

dict_keys(['data', 'error_code', 'error_description'])

In [6]:
data['data'].keys()

dict_keys(['symbol', 'column', 'item'])

# 解析获取信息

In [7]:
df=pd.DataFrame(data=data["data"]['item'],columns=data['data']['column'])

In [8]:
df.head().stack().unstack(0)

,0,1,2,3,4
timestamp,1552233600000,1552320000000,1552406400000,1552492800000,1552579200000
volume,115765077,141179492,126942716,105746790,104540603
open,12.1674,12.3553,12.2069,12.1971,12.2069
high,12.3553,12.5037,12.4147,12.4839,12.5433
low,11.9201,12.108,11.9992,12.1377,12.2069
close,12.1872,12.2267,12.2366,12.296,12.3652
chg,0.0195,0.039,0.0098,0.06,0.0689
percent,0.16,0.32,0.08,0.49,0.56
turnoverrate,0.67,0.82,0.73,0.61,0.6
amount,1.416e+09,1.7578e+09,1.56839e+09,1.31637e+09,1.31132e+09


In [9]:
df.timestamp=df.timestamp.apply(lambda x:datetime.datetime.fromtimestamp(x/1000).date())

In [10]:
df.head().stack().unstack(0)

,0,1,2,3,4
timestamp,2019-03-11,2019-03-12,2019-03-13,2019-03-14,2019-03-15
volume,115765077,141179492,126942716,105746790,104540603
open,12.1674,12.3553,12.2069,12.1971,12.2069
high,12.3553,12.5037,12.4147,12.4839,12.5433
low,11.9201,12.108,11.9992,12.1377,12.2069
close,12.1872,12.2267,12.2366,12.296,12.3652
chg,0.0195,0.039,0.0098,0.06,0.0689
percent,0.16,0.32,0.08,0.49,0.56
turnoverrate,0.67,0.82,0.73,0.61,0.6
amount,1.416e+09,1.7578e+09,1.56839e+09,1.31637e+09,1.31132e+09


# 封装爬虫

In [11]:
class XueQiu(object):
    def __init__(self,token):
        self._token=token
    def get_kline(self,code:str='SZ000001',end:str='2019-11-04',count:int=100):
        end=int(datetime.datetime.strptime(end,'%Y-%m-%d').timestamp()*1000)
        url=('https://stock.xueqiu.com/v5/stock/chart/kline.json?'+
             'symbol={code}&begin={end}&period=day&type=before&count=-{count}'+
             '&indicator=kline,pe,pb,ps,pcf,market_capital,agt,ggt,balance'
        ).format(code=code,end=end,count=count)
        res=requests.get(
            url=url,
            headers={
                'User-Agent':'pyqi',
                'Cookie':'xq_a_token={token}'.format(token=self._token),
            },
        )
        data=json.loads(res.text)
        df=pd.DataFrame(data=data["data"]['item'],columns=data['data']['column'])
        df.timestamp=df.timestamp.apply(lambda x:datetime.datetime.fromtimestamp(x/1000).date())
        
        return df

In [12]:
xq=XueQiu('1d1413ef0b90ad65cb650d1d8651ff2941335e2e')
df=xq.get_kline(code='SZ000002',end='2019-10-08',count=10000)
df.head()

,timestamp,volume,open,high,low,close,chg,percent,turnoverrate,amount,...,ps,pcf,market_capital,balance,hold_volume_cn,hold_ratio_cn,net_volume_cn,hold_volume_hk,hold_ratio_hk,net_volume_hk
0,1991-01-03,4500,0.1078,0.1078,0.1078,0.1078,-0.0017,-1.54,0.02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1,1991-01-04,6800,0.1072,0.1072,0.1072,0.1072,-0.0006,-0.52,0.02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,1991-01-07,7300,0.1084,0.1084,0.1084,0.1084,0.0011,1.05,0.03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
3,1991-01-09,3200,0.1072,0.1072,0.1072,0.1072,-0.0011,-1.03,0.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
4,1991-01-10,300,0.1067,0.1067,0.1067,0.1067,-0.0006,-0.52,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


# pyecharts

In [13]:
legend_opts=opts.LegendOpts(
    is_show=True,
    pos_left='right',
)
tooltip_opts=opts.TooltipOpts(
    is_show=True,
    trigger='axis',
    axis_pointer_type='cross',
    trigger_on = "mousemove|click",      
    textstyle_opts=opts.TextStyleOpts(
        font_size=10,
    ),    
)
tooltip_opts.opts['position']=JsCode(
    js_code="""
function (pos, params, el, elRect, size) {
    var obj = {top: 10};
    obj[['left', 'right'][+(pos[0] < size.viewSize[0] / 2)]] = 5;
    return obj;
}    
    """
)
axispointer_opts=opts.AxisPointerOpts(
    is_show=True,
    link={'xAxisIndex': 'all',},
)
def get_axis_opts(
    split:bool=False,
    tick_label:bool=False,
):
    return opts.AxisOpts(
        is_scale=True,# 不必显示零刻度
        is_show=True,# 显示轴
        splitarea_opts=opts.SplitAreaOpts(
            is_show=split,
            areastyle_opts=opts.AreaStyleOpts(opacity=1,)
        ),
        splitline_opts=opts.SplitLineOpts(
            is_show=split,
        ),
        axistick_opts=opts.AxisTickOpts(is_show=tick_label,),
        axislabel_opts=opts.AxisLineOpts(is_show=tick_label,),
    )
def get_line(
    xaxis:pd.Series,
    yaxis,
    x_tick_label:bool=False,    
)->Line:
    line=Line().add_xaxis(xaxis_data=xaxis.values.tolist(),)
    for i in yaxis:
        line=line.add_yaxis(
            series_name=i.name,
            y_axis=i.values.tolist(),
        )
    line=(
        line
        .set_series_opts(
            symbol='none',
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            xaxis_opts=get_axis_opts(split=False,tick_label=x_tick_label),
            yaxis_opts=get_axis_opts(split=True ,tick_label=True),
            legend_opts=legend_opts,            
        )
    )
    return line
def get_bar(
    xaxis:pd.Series,
    yaxis,
    x_tick_label:bool=False,    
)->Bar:
    bar=Bar().add_xaxis(xaxis_data=xaxis.values.tolist(),)
    for i in yaxis:
        bar=bar.add_yaxis(
            series_name=i.name,
            yaxis_data=i.values.tolist(),
        )
    bar=(
        bar
        .set_series_opts(
            symbol='none',
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            xaxis_opts=get_axis_opts(split=False,tick_label=x_tick_label),
            yaxis_opts=get_axis_opts(split=True ,tick_label=True),
            legend_opts=legend_opts,            
        )
    )
    return bar
def get_scatter(
    xaxis:pd.Series,
    yaxis,
    x_tick_label:bool=False,    
)->Scatter:
    bar=Scatter().add_xaxis(xaxis_data=xaxis.values.tolist(),)
    for i in yaxis:
        bar=bar.add_yaxis(
            series_name=i.name,
            y_axis=i.values.tolist(),
            symbol_size=5,
        )
    bar=(
        bar
        .set_series_opts(
            symbol='circle',
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            xaxis_opts=get_axis_opts(split=False,tick_label=x_tick_label),
            yaxis_opts=get_axis_opts(split=True ,tick_label=True),
            legend_opts=legend_opts,            
        )
    )
    return bar
def get_kline(
    xaxis:pd.Series,
    yaxis:pd.DataFrame,
    x_tick_label:bool=False,
    signal:pd.Series=None,# 信号序列。-1：空头信号；0：无信号，1：多头信号
    signal_shift_num:int=1,
)->Kline:
    # 获得透明序列
    def _opacity(signal,signal_shift_num):
        return (
            pd
            .concat([signal.shift(-i) for i in range(signal_shift_num)],axis=1)# 合并当前与后面N个周期的信号
            .apply(lambda x:np.sign(x.sum()),axis=1)# 如有信号则当前设置为有信号状态
        )
        
    y_axis=yaxis.loc[:,['open','close','low','high']].values.tolist()# 注意先后顺序
    opacity=None if signal is None else _opacity(signal,signal_shift_num)# 获得透明序列
    
    # 信号突出显示
    def _yaxis(x:list,y:float):
        return {
            'value':x,
            'itemStyle':{'opacity':1 if abs(y)==1 else .2}
        }
    y_axis=(
        y_axis if signal is None else  
        list(map(lambda x,y:_yaxis(x,y),y_axis,opacity))
    )
    
    # 信号标记   
    def _mark(xaxis,yaxis,signal):
        (rotate,y)=(180,yaxis.low) if signal.sum()>0 else (0,yaxis.high)
        mark=pd.concat([xaxis,y,signal],axis=1)
        mark=mark.loc[mark.iloc[:,2].abs()==1,:].iloc[:,:2].values.tolist()
        return rotate,mark
    (rotate,mark)=(0,[]) if signal is None else _mark(xaxis,yaxis,signal)
    markpoint_opts=opts.MarkPointOpts( 
        symbol_size=20,
        data=[
            opts.MarkPointItem(
                coord=i,
            )
            for i in mark
        ],
    )
    # 旋转角度
    markpoint_opts.opts['symbolRotate']=rotate
    # 颜色
    markpoint_opts.opts['itemStyle']={
        'normal':{'color':'#0098d9' if rotate==0 else '#c12e34',},
    }
    kline=(
        Kline()# K线对象实例化
        .add_xaxis(xaxis_data=xaxis.values.tolist(),)# 添加X轴数据
        .add_yaxis(# 添加Y轴数据
            series_name='K',
            y_axis=y_axis,
            itemstyle_opts=opts.ItemStyleOpts(# K线红绿柱的颜色配置          
                color="#ec0000",
                color0="#00da3c",
                border_color="#ec0000",
                border_color0="#00da3c",
            ),
        ) 
        .set_series_opts(
            markpoint_opts=markpoint_opts,
        )
        .set_global_opts(# 全局配置
            xaxis_opts=get_axis_opts(split=False,tick_label=x_tick_label),
            yaxis_opts=get_axis_opts(split=True ,tick_label=True),
            legend_opts=legend_opts,
        )        
    )
    return kline
def get_datazoom_opts(
    range_start,
    range_end,
    range_len,
    pos_bottom,
    chart_num,
):
    return [# 注意先后顺序
        opts.DataZoomOpts(
            is_show=True,
            type_='inside',
            xaxis_index=[i for i in range(chart_num)],
            range_start=100*(range_start/range_len),
            range_end=100*(range_end/range_len),
        ),
        opts.DataZoomOpts(
            is_show=True,
            type_='slider',
            xaxis_index=[i for i in range(chart_num)],
            pos_bottom=pos_bottom,
        ),# here ok
    ]
def pos_calc(a,b):
    return '{}%'.format(float(a.replace("%",""))+float(b.replace("%","")))
def get_grid_opts(pos_top,height):
    grid_opts=opts.GridOpts(
        pos_left='3%',
        pos_right='0%',
    )
    grid_opts.update(top=pos_top,height=height)   
    return grid_opts

def get_grid(
    charts,heights,range_start,range_end,height='400px'
)->Grid:
    tops=list(np.array(heights).cumsum())
    tops.insert(0,0)
    tops=["{}%".format(i) for i in tops]
    heights=["{}%".format(i) for i in heights]
    grid=Grid(init_opts=opts.InitOpts(width='100%',height=height))
    charts[0].options.update(
        dataZoom=get_datazoom_opts(
            range_start=range_start,
            range_end=range_end,
            range_len=len(charts[0].options['xAxis'][0]['data']),
            chart_num=len(charts),
            pos_bottom=pos_calc('100%','-'+tops[1]),
        ),
        tooltip=tooltip_opts,
        axisPointer=axispointer_opts,
    )
    for i in range(len(charts)):
        chart=charts[i]
        chart.options['legend'][0].update(top=tops[i],left='right',)
        chart.options['xAxis'][0]['axisPointer']=opts.AxisPointerOpts(
            is_show=True,
            type_='shadow',
            label=opts.LabelOpts(
                is_show=False,
            )
        )
        grid=grid.add(
            chart=chart,
            grid_opts=get_grid_opts(pos_top=tops[i],height=heights[i],),
        )
    chart.options['xAxis'][0]['axisPointer'].opts['label'].opts['show']=True
    return grid

# 绘制K线

In [14]:
dx=df.loc[:,['timestamp','open','high','low','close']]
dx.rename(inplace=True,columns={'timestamp':'date'})
dx.head()

,date,open,high,low,close
0,1991-01-03,0.1078,0.1078,0.1078,0.1078
1,1991-01-04,0.1072,0.1072,0.1072,0.1072
2,1991-01-07,0.1084,0.1084,0.1084,0.1084
3,1991-01-09,0.1072,0.1072,0.1072,0.1072
4,1991-01-10,0.1067,0.1067,0.1067,0.1067


In [15]:
# sample
d=dx
signal=pd.Series(np.zeros(d.shape[0]))
signal[::100]=-1
grid=get_grid(
    charts=[
        get_kline(xaxis=d.date,yaxis=d,x_tick_label=True),
    ],
    heights=[95],
    range_start=d.date.shape[0]-50,
    range_end=d.date.shape[0],
    height='300px',
)

# jupyter lab
grid.load_javascript()

In [16]:
grid.render_notebook()

# 保存数据

In [17]:
df.shape

(6773, 24)

In [18]:
df.to_csv('000002.csv',index=False)